In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon as wilcoxon
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

# meta analysis functions
import sys
sys.path.append('/code/')
from util import analysis
from util import plotting
from util import util_continuous as utils


In [ ]:
PATH = 'data/'

In [ ]:
with open(PATH + 'encoder-estimation-data/encoder-decoder-data.pickle', 'rb') as handle:
    encoder, encoder_r2, idx_dict, pos_vel_model, pos, dec_vels, decoders = pickle.load(handle)
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']


with open(PATH + 'trial-related-data/times-decoder-update.pickle','rb') as handle:
    update_ix, update_times = pickle.load(handle)
# load the order of the trials in terms of when the subject saw the trial
with open(PATH + 'trial-related-data/decoder_trials_in_order.pickle','rb') as handle:
    trials_list, conds_trial_list = pickle.load(handle)

In [ ]:
# check the encoder shape to match as expected
assert(encoder.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, utils.n_ch, 9))
encoder_linear = encoder[:, :, :, :, :, :-1]

# remove the affine term from the encoder
assert(encoder_linear.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, utils.n_ch, 8))


In [ ]:
import matplotlib.ticker as ticker

label_size = 6
## SETUP THE FIGURE HERE
## HAVE TO RE-REUN FROM HERE TO "CLEAR" THE PLOT
fig_encdec = plt.figure(figsize = (6.5, 4), layout='constrained') # set the total figure size
mosaic = """
    aabb.eeff
    ccdd.gghh
    .........
    iijj.mmnn
    kkll.oopp
    """

# set up the axes
ax_dict = fig_encdec.subplot_mosaic(mosaic)
for ii in ax_dict:
    plotting.remove_and_set_axes(ax_dict[ii], bottom=True, left=True)
    ax_dict[ii].tick_params(axis='both', which='major', labelsize = label_size)
    ax_dict[ii].tick_params(axis='both', which='minor', labelsize = label_size)
    # set the x-ticks at multiples of 1
    ax_dict[ii].xaxis.set_major_locator(ticker.MultipleLocator(base=5))
    ax_dict[ii].xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
fig_encdec.patch.set_facecolor('white')


ax_dict['a'].set_title("$D \cdot F_0$")


ax_dict['a'].set_ylabel("(0, 0)")
ax_dict['b'].set_ylabel("(0, 1)")
ax_dict['c'].set_ylabel("(1, 0)")
ax_dict['d'].set_ylabel("(1, 1)")

ax_dict['e'].set_title("$D \cdot F_1$")

ax_dict['e'].set_ylabel("(0, 0)")
ax_dict['f'].set_ylabel("(0, 1)")
ax_dict['g'].set_ylabel("(1, 0)")
ax_dict['h'].set_ylabel("(1, 1)")

ax_dict['i'].set_title("$D \cdot B_0$")

ax_dict['i'].set_ylabel("(0, 0)")
ax_dict['j'].set_ylabel("(0, 1)")
ax_dict['k'].set_ylabel("(1, 0)")
ax_dict['l'].set_ylabel("(1, 1)")

ax_dict['m'].set_title("$D \cdot B_1$")

ax_dict['m'].set_ylabel("(0, 0)")
ax_dict['n'].set_ylabel("(0, 1)")
ax_dict['o'].set_ylabel("(1, 0)")
ax_dict['p'].set_ylabel("(1, 1)")


In [ ]:
## set up the time axis
t_upd = update_times[1:-1]

In [ ]:
idx_dict

In [ ]:
# plots a, b, c, d
# target position contribution (D * F0)
enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_pos_idx']] 

# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))


## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    # dec1 = Ws_block1[key][:, update_ix][:, 1:-1] # make the shape the same as the encoder - 8 x 17 x 2 x 64
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1
    
    # block 1
    b = 1
    # dec2 = Ws_block2[key][:, update_ix][:, 1:-1]
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2

ed_all = np.mean(enc_dec, axis = (0, 2))

# take the average of the slow conditions and the fast conditions
ed_all_slow = np.mean(enc_dec[:, :, utils.slow], axis = (0, 2))
ed_all_fast = np.mean(enc_dec[:, :, utils.fast], axis = (0, 2))
# shape is that each subject has a slow/fast decoder-encoder at each timepoint to graph
assert(ed_all_slow.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))
assert(ed_all_fast.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))

# learning rate = low
# a, b
# c, d
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['a'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['b'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 0],
                          axis = 0, ax = ax_dict['c'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['d'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['a'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['b'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 0], 
                          axis = 0, ax = ax_dict['c'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['d'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

# fig_encdec.legend(['slow', '', 'fast'])


# set idealized lines
ax_dict['a'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)
ax_dict['b'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)
ax_dict['c'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)
ax_dict['d'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)

ax_dict['a'].get_shared_y_axes().join(ax_dict['a'], ax_dict['b'], ax_dict['c'], ax_dict['d'])




# fig.suptitle("Target Velocity Contribution")
fig_encdec


In [ ]:
# plots e,f,g,h
# target velocity contribution (D * F1)
enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_vel_idx']] 

# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))


## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    # dec1 = Ws_block1[key][:, update_ix][:, 1:-1] # make the shape the same as the encoder - 8 x 17 x 2 x 64
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1
    
    # block 1
    b = 1
    # dec2 = Ws_block2[key][:, update_ix][:, 1:-1]
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2

ed_all = np.mean(enc_dec, axis = (0, 2))

# take the average of the slow conditions and the fast conditions
ed_all_slow = np.mean(enc_dec[:, :, utils.slow], axis = (0, 2))
ed_all_fast = np.mean(enc_dec[:, :, utils.fast], axis = (0, 2))
# shape is that each subject has a slow/fast decoder-encoder at each timepoint to graph
assert(ed_all_slow.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))
assert(ed_all_fast.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))

# learning rate = low
# e, f
# g, h
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['e'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['f'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 0],
                          axis = 0, ax = ax_dict['g'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['h'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['e'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['f'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 0], 
                          axis = 0, ax = ax_dict['g'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['h'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

# fig_encdec.legend(['slow', '', 'fast'])

# set idealized lines
ax_dict['e'].axhline(y=1,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)
ax_dict['f'].axhline(y=0,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)
ax_dict['g'].axhline(y=0,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)
ax_dict['h'].axhline(y=1,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)

# share y-axis
ax_dict['e'].get_shared_y_axes().join(ax_dict['e'], ax_dict['f'], ax_dict['g'], ax_dict['h'])


# fig.suptitle("Target Velocity Contribution")
fig_encdec


In [ ]:
# plots i,j,k,l
# target velocity contribution (D * B0)
enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_pos_err_idx']] 

# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))


## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    # dec1 = Ws_block1[key][:, update_ix][:, 1:-1] # make the shape the same as the encoder - 8 x 17 x 2 x 64
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1
    
    # block 1
    b = 1
    # dec2 = Ws_block2[key][:, update_ix][:, 1:-1]
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2

ed_all = np.mean(enc_dec, axis = (0, 2))

# take the average of the slow conditions and the fast conditions
ed_all_slow = np.mean(enc_dec[:, :, utils.slow], axis = (0, 2))
ed_all_fast = np.mean(enc_dec[:, :, utils.fast], axis = (0, 2))
# shape is that each subject has a slow/fast decoder-encoder at each timepoint to graph
assert(ed_all_slow.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))
assert(ed_all_fast.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))

# learning rate = slow
# i, j
# k, l
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['i'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['j'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 0],
                          axis = 0, ax = ax_dict['k'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['l'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

# learning rate = fast
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['i'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['j'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 0], 
                          axis = 0, ax = ax_dict['k'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['l'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

# fig_encdec.legend(['slow', '', 'fast'])


# set idealized lines
ax_dict['j'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)
ax_dict['k'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)

# share y-axis
ax_dict['j'].get_shared_y_axes().join(ax_dict['i'], ax_dict['j'], ax_dict['k'], ax_dict['l'])


fig_encdec


In [ ]:
# plots m,n,o,p
# target velocity contribution (D * B1)
enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_vel_err_idx']] 

# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))


## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    # dec1 = Ws_block1[key][:, update_ix][:, 1:-1] # make the shape the same as the encoder - 8 x 17 x 2 x 64
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1
    
    # block 1
    b = 1
    # dec2 = Ws_block2[key][:, update_ix][:, 1:-1]
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2

ed_all = np.mean(enc_dec, axis = (0, 2))

# take the average of the slow conditions and the fast conditions
ed_all_slow = np.mean(enc_dec[:, :, utils.slow], axis = (0, 2))
ed_all_fast = np.mean(enc_dec[:, :, utils.fast], axis = (0, 2))
# shape is that each subject has a slow/fast decoder-encoder at each timepoint to graph
assert(ed_all_slow.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))
assert(ed_all_fast.shape == (utils.n_keys, len(update_ix) - 2, 2, 2))

# learning rate = slow
# m, n
# o, p
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['m'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['n'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 0],
                          axis = 0, ax = ax_dict['o'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_slow[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['p'], color = 'blue', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

# learning rate = fast
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 0], 
                          axis = 0, ax = ax_dict['m'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 0, 1], 
                          axis = 0, ax = ax_dict['n'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 0], 
                          axis = 0, ax = ax_dict['o'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)
plotting.plot_time_domain(t_upd/60, ed_all_fast[:, :, 1, 1], 
                          axis = 0, ax = ax_dict['p'], color = 'red', ls='-o', alpha = 0.4, lw=1, remove_axes = False)

fig_encdec.legend(['slow', '', 'fast', ''],loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))


# idealized values
ax_dict['n'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)
ax_dict['o'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)

# share y-axis
ax_dict['n'].get_shared_y_axes().join(ax_dict['m'], ax_dict['n'], ax_dict['o'], ax_dict['p'])



fig_encdec


In [ ]:
## need to re-adjust the axes

ax_dict['a'].set_ylim(-.5, .5)
ax_dict['a'].get_shared_y_axes().join(ax_dict['a'], ax_dict['b'], ax_dict['c'], ax_dict['d'])

ax_dict['e'].set_ylim(-.2, 1)
ax_dict['e'].get_shared_y_axes().join(ax_dict['e'], ax_dict['f'], ax_dict['g'], ax_dict['h'])

ax_dict['j'].set_ylim(-.5, .5)
ax_dict['j'].get_shared_y_axes().join(ax_dict['i'], ax_dict['j'], ax_dict['k'], ax_dict['l'])

ax_dict['n'].set_ylim(-1, 0.2)
ax_dict['n'].get_shared_y_axes().join(ax_dict['m'], ax_dict['n'], ax_dict['o'], ax_dict['p'])

fig_encdec

In [ ]:
image_format = 'pdf' # e.g .png, .svg, etc.
image_name = 'fig4-dec-enc-learning-rates-mean.pdf'
PATH = '/results/'
fig_encdec.savefig(PATH + image_name, format=image_format, dpi=300)